In [1]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")

In [4]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de référance à des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        if(result["itemLabel"]["value"][0]!="Q"):                                   
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [5]:
listofidqHuman = ['Q22808320', 'Q571', 'Q31629', 'Q188451', 'Q192581', 'Q11424', 'Q11410', 'Q2095', 'Q3624078']


In [6]:
description=[]
for i in range(len(listofidqHuman)):
    description.append(research(listofidqHuman[i]))

In [7]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [8]:
model_path = "./models/"

In [9]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [10]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [11]:
import unicodedata

In [12]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [13]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format(model_path+'GoogleNews-vectors-negative300.bin', binary=True) 

In [14]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport', 'book', 'film']

In [15]:
class Person:
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.book = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8')
        self.film =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')
        self.nameofspeaker = ""
        
    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteBook | '+self.book
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFilm | '+self.film
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\njob: '+self.job+'\nbook: '+self.book+'\nsport: '+self.sport+'\nmusic: '+self.music+'\nfilm: '+self.film+'\ngame: '+self.game+'\nfood: '+self.food+'\n')
        
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def respondToFood(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFood()
        print(text)

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.optimizeJob(self.job)+'.'
        return text
    
    def optimizeJob(self, job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
            elif split[i]=='therapy':
                split[i]='therapist'
            elif split[i]=='supervision':
                split[i]='supervisor'
            elif split[i]=='scouting':
                split[i]='scout'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def respondToJob(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourJob()
        print(text)

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text

    def respondToMusic(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteMusic()
        print(text)
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text

    def respondToGame(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteGame()
        print(text)
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,9)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text

    def respondToSport(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteSport()
        print(text)
        
    def tellYourFavouriteFilm(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite film is '
        elif rand==1:
            text+='I really liked the film '
        elif rand==2:
            text+='I really enjoyed watching the film '
        elif rand==3:
            text+='I like films like '
        text+=self.film
        if rand<4:
            text+='.'
        elif rand==4:
            text+=' was the film that I liked most.'
        return text
    
    def startTopicFilm(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFilm()+' '
        rand=random.randint(0,3)
        if rand==0:
            text+='What is your favourite film?'
        elif rand==1:
            text+='Which film did you like most?'
        elif rand==2:
            text+='Which films do watch?'
        elif rand==3:
            text+='Which film did you enjoy most?'
        print(name+text)
        return text
    
    def respondToFilm(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteFilm()
        print(text)
        
    def tellYourFavouriteBook(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite book is '
        elif rand==1:
            text+='I really like the book '
        elif rand==2:
            text+='I enjoyed reading the book '
        elif rand==3:
            text+='I am currently reading '
        elif rand==4:
            text+='I like the book '
        elif rand==5:
            text+='I find '
        text+=self.book
        if rand<3:
            text+='.'
        elif rand==3:
            text+=' and i really like it.'
        elif rand==4:
            text+=' a lot.'
        elif rand==5:
            text+=' is a pretty good book.'
        return text
    
    def startTopicBook(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteBook()+' '
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite book?'
        elif rand==1:
            text+='What do you like to read?'
        elif rand==2:
            text+='Which kind of books do you like?'
        elif rand==3:
            text+='Do you read books?'
        elif rand==4:
            text+='Which book do you like most?'
        print(name+text)
        return text
    
    def respondToBook(self):
        text=self.name.split(' ')[0]+':   '
        text+=self.tellYourFavouriteBook()
        print(text)

    def respond(self, message):
        phrases = []
        questions = []
        words = ""
        for letter in message:
            if(letter == '.' or letter == '!'):
                phrases.append(words)
                print(words + " -> is a phrase")
                words = ""
            elif(letters == '?'):
                questions.append(words)
                print(words + " -> is a question")
                words = ""
            else:    
                words+=letters     
        
        SWITCH_DICT={
            'sport':self.respondToSport,
            'music':self.respondToMusic,
            'job':self.respondToJob,
            'game':self.respondToGame,
            'food':self.respondToFood,
            'film':self.respondToFilm,
            'book':self.respondToBook
        }
        topicsquestions = []
        if(len(questions)!= 0):
            for question in questions:
                guessedTopic=self.guessTopic(question)
                if not guessedTopic==None:
                    topicsquestions.append(guessedTopic)
        if(len(topicsquestions)!=0):            
            SWITCH_DICT.get(topicsquestions[random.randint(0,len(topicsquestions)-1)])()
        topicsphrases = []    
        if(len(phrases) != 0):
            for phrase in phrases:
                guessedTopic=self.guessTopic(phrase)
                if not guessedTopic==None:
                    topicsphrases.append(guessedTopic)
        if(len(topicsphrases)!=0):            
            SWITCH_DICT.get(topicsphrases[random.randint(0,len(topicsphrases)-1)])()
        if(len(topicsquestions)==0 and len(topicsphrases)==0):     
            print("I don't know what you are speaking about. :( \n Do you wan't to speak about sport, Films, music ?")
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            #print(word)
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    #print(distance)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        elif openTopics[rand]=='film':
            doneTopics.append('film')
            return self.startTopicFilm()
        elif openTopics[rand]=='book':
            doneTopics.append('book')
            return self.startTopicBook()
        else:
            return None

In [16]:
person = Person()
person.respond("my name is matthis . What sport do you like ? i love Sandwiches.")

my name is matthis  -> is a phrase
 What sport do you like  -> is a question
 i love Sandwiches -> is a phrase
Zoltan:   I like playing dancesport.
Zoltan:   My favourite dish is Soto Madura.


In [17]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [18]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
        person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [19]:
import time
def startChatInteractive(persons):
    for person in persons: 
        person.introduce()
        time.sleep(2)
    while True:
        message=input('You: ')
        if message=='bye':
            return
        if not message=='':
            random.shuffle(persons)
            for person in persons:
                time.sleep(2)
                res=person.respond(message)
                if res==-1:
                    break

In [20]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [21]:
myPersons=generatePersons(4)

In [22]:
startChat(myPersons, 5)           

Francisco:   Hello, I am Francisco Rodriguez, I am 31 years old and I live in Vietnam.
Zoltan:   Good day, I am Zoltan Varga, I'm 56 years old and I'm from Chile.
Julius:   Hey, I'm Julius Friedlander, I'm 40 years old and I am from Estonia.
Arthur:   Hello, I am Arthur Paget, I'm 75 years old and I come from Croatia.
Julius:   I really enjoyed watching the film The Lives of a Bengal Lancer. Which film did you enjoy most?
I really enjoyed watching the film The Lives of a Bengal Lancer -> is a phrase
 Which film did you enjoy most -> is a question
Arthur:   I really enjoyed watching the film Town & Country.
Arthur:   I really liked the film Town & Country.
I really enjoyed watching the film The Lives of a Bengal Lancer -> is a phrase
 Which film did you enjoy most -> is a question
Zoltan:   I really enjoyed watching the film The Banquet.
Zoltan:   I like films like The Banquet.
I really enjoyed watching the film The Lives of a Bengal Lancer -> is a phrase
 Which film did you enjoy most 

In [23]:
startChatInteractive(myPersons)

Francisco:   Hi, my name is Francisco Rodriguez, I am 31 years old and I'm from Vietnam.
Zoltan:   Hello, my name is Zoltan Varga, I'm 56 years old and I am from Chile.
Julius:   Hi, I'm Julius Friedlander, I am 40 years old and I'm from Estonia.
Arthur:   Hello, I'm Arthur Paget, I'm 75 years old and I live in Croatia.
You: bye


In [24]:
generateDescriptions(myPersons)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Francisco Rodriguez, who is from Vietnam, has a career as a fighter and has 31 years to his credit.
The eunuch official title of the Francisco employee is the eunuch official.
Danube-bridges from the Black Sea are the favourite books of Francisco.
Inline skating is a favourite sport of Francisco.
The favourite music of Francisco is the music of Croatia.
The favourite Film of Francisco is I Vitelloni.
Paper Chase is the favourite of Fancisco.
The tahini roll is a favourite of Francisco.


The Zoltan Varga is in the country of Chile and has a height of 5.
Zaoltan is a contractor.
Zoltan's favourite book is time of Contempt.
The favourite sport of Zoltan is Aquaplaning.
regueifa is the favourite of Zoltan.
The favourite Film of Zoltan is The Bant.
The favourite games of Zoltan are strip poker.
The favourite food of Zaoltan is nasi tumpang.


Julius Friedlander, who is from the country of Estonia, has a capacity of 40.
Julius is a member of the nihon-buyo dancer.
The book "Wolfs: The Voice

In [25]:
for person in myPersons:
    person.showProperties()

name: Francisco Rodriguez
age: 31
country: Vietnam
job: eunuch official
book: Danube-bridges from the Black Forest to the Black Sea
sport: inline skating
music: music of Croatia
film: I Vitelloni
game: Paper Chase
food: tahini roll

name: Zoltan Varga
age: 56
country: Chile
job: contractor
book: Time of Contempt
sport: Aquaplaning sport
music: regueifa
film: The Banquet
game: strip poker
food: nasi tumpang

name: Julius Friedlander
age: 40
country: Estonia
job: nihon-buyo dancer
book: Whispers: The Voices of Paranoia
sport: sled dog racing
music: Celtic fusion
film: The Lives of a Bengal Lancer
game: boule bretonne des Cotes-d'Armor
food: Peynir helva

name: Arthur Paget
age: 75
country: Croatia
job: horse trader
book: Fadl al-kilab ala man labisa al-thiyab
sport: Laser-Run
music: pasodoble
film: Town & Country
game: London
food: Khubz

